<a href="https://colab.research.google.com/github/HrithikNambiar/COVID-19-Cough-Detection/blob/main/CCS_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install liac-arff
!pip install -U PyYAML

  Created wheel for liac-arff: filename=liac_arff-2.5.0-cp37-none-any.whl size=11732 sha256=1e32e3870eaa5290ff7b5126eb183a1e803fafc90cd080acaebaaff86187b6b6
  Stored in directory: /root/.cache/pip/wheels/77/8d/b4/8bfce5beea9a3496cc15b24961876adb7b6e2912ff09164179
Successfully built liac-arff
     |████████████████████████████████| 645kB 8.6MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
cd '/content/drive/MyDrive/ComParE2021'

/content/drive/.shortcut-targets-by-id/1a6R2J7y9rkZWY_SyloMCwmm8qdQAliCz/ComParE2021


In [ ]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from xgboost import XGBClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.base import clone
# from sklearn.pipeline import Pipeline
from sklearn.utils import resample
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, recall_score, make_scorer
from joblib import Parallel, delayed
import pandas as pd
import scipy
import os, yaml
import json
import sys
import arff
import numpy as np
from tqdm import tqdm
from glob import glob

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
FEATURE_BASE='./features_2'  #path to the directory containing features

RANDOM_SEED = 42

GRID = [
    {'scaler': [StandardScaler(), None],
     'estimator': [SVC(probability=True )],
    #  'estimator__loss': ['squared_hinge'],
     'estimator__kernel' : ['linear', 'rbf'],
     'estimator__C': np.logspace(-1, -5, num=5),
     'estimator__class_weight': ['balanced', None],
     'estimator__max_iter': [10000]
     }
]

PIPELINE = Pipeline([('scaler', None), ('sampling', RandomUnderSampler()), ('estimator', SVC())])

# GRID = [
#     {'scaler': [StandardScaler(), MinMaxScaler(), None],
#      'estimator': [RandomForestClassifier(random_state=RANDOM_SEED)],
#     #  'estimator__n_estimators': [200, 400, 600, 800, 1000],
#      }
# ]

# PIPELINE = Pipeline([('scaler', None), ('sampling', SMOTE()), ('estimator', RandomForestClassifier())])

# clf1 = SVC(random_state=RANDOM_SEED, probability=True)
# clf2 = RandomForestClassifier(random_state=RANDOM_SEED)
# clf3 = XGBClassifier(random_state=RANDOM_SEED)
# clf4 = MLPClassifier(hidden_layer_sizes=(256, 256, 256,))

# eclf = VotingClassifier(estimators=[('svm', clf1), ('rf', clf2), ('clf3', clf3), ('nn', clf4)],voting='soft')

# GRID = [
#     {'scaler': [MinMaxScaler()],
#      'estimator': [eclf],
#     #  'estimator__n_estimators': [200, 400, 600, 800, 1000],
#      }
# ]

# PIPELINE = Pipeline([ ('scaler', None), ('sampling', RandomUnderSampler()), ('estimator', eclf)])


In [ ]:
def make_dict_json_serializable(meta_dict: dict) -> dict:
    cleaned_meta_dict = meta_dict.copy()
    for key in cleaned_meta_dict:
        if type(cleaned_meta_dict[key]) not in [str, float, int, np.float]:
            cleaned_meta_dict[key] = str(cleaned_meta_dict[key])
    return cleaned_meta_dict

def bootstrap(best_estimator, X, y, test_X, test_y, random_state):
    estimator = clone(best_estimator)
    sample_X, sample_y = resample(X, y, random_state=random_state)
    estimator.fit(sample_X, sample_y)
    _preds = estimator.predict(test_X)
    return recall_score(test_y, _preds, average='macro')

In [ ]:
def run_predictions_devel(model, feature_folder, results_folder): 
  devel_file = glob(os.path.join(dirpath, 'val.*'))[0]
  if "auDeep" in dirpath:
      label_index = -2  #because last column of auDeep feature is the label
  else:
      label_index = -1

  if devel_file.endswith('.arff'):
    with open(devel_file) as f:
        arff_data = arff.load(f)
        devel_names = np.array(arff_data['data'])[:, 0]
        devel_X = np.array(arff_data['data'])[:, 1:label_index].astype(np.float32)
        devel_y = np.array(arff_data['data'])[:, label_index].astype(str)
  else:
    devel_df = pd.read_csv(devel_file)
    devel_names = devel_df.values[:, 0]
    devel_X = devel_df.values[:, 1:label_index].astype(np.float32)
    devel_y = devel_df.values[:, label_index].astype(str)
  
  preds = model.predict(devel_X)

  prob = model.predict_proba(devel_X)
  pos_prob = prob[:, 1]

  df_predictions = pd.DataFrame({'filename': devel_names.tolist(), 'prediction': preds.tolist(), 'pos_prob': pos_prob.tolist()})
  df_predictions.to_csv(os.path.join('/content/', 'devel.predictions.csv'), index=False)

  df = pd.read_csv("/content/devel.predictions.csv")
  temp_dict = {}

  for i in range(len(df)):
    f = df['filename'][i][:9]+'.wav'
    c = df['prediction'][i]
    pos = df['pos_prob'][i]
    if f in temp_dict.keys():
      temp_dict[f].append([c, pos])
    else:
      temp_dict[f] = []
      temp_dict[f].append([c, pos])

  fnames = []
  preds = []
  for i in temp_dict.keys():
    cnt_pos = 0
    cnt_neg = 0
    probs = []
    for i in temp_dict[i]:
      if i[0] == 'positive':
        cnt_pos += 1
      else:
        cnt_neg += 1
      probs.append(i[1])
      
    if cnt_pos > cnt_neg: 
      preds.append('positive')
    elif cnt_neg > cnt_pos:
      preds.append('negative')
    else:
      temp = np.mean(probs)
      if temp >= 0.5:
        preds.append('positive')
      else:
        preds.append('negative')
    fnames.append(i)

  gt_df = pd.read_csv('./dist/lab/devel.csv')
  gt_preds = list(gt_df['label'])
  gt_df.head()

  uar = recall_score(gt_preds, preds, average='macro')
  print(uar)

  cm = confusion_matrix(gt_preds, preds)
  print(cm)

  df_predictions = pd.DataFrame({'filename': fnames, 'prediction': preds})
  df_predictions.to_csv(os.path.join(results_folder, 'devel.predictions.csv'), index=False)

In [ ]:
def run_svm(feature_folder, results_folder, params):
    train_file = glob(os.path.join(feature_folder, 'train.*'))[0] 
    # devel_file = glob(os.path.join(feature_folder, 'devel.*'))[0]
    devel_file = glob(os.path.join(feature_folder, 'val.*'))[0]
    test_file = glob(os.path.join(feature_folder, 'test.*'))[0]
    if "auDeep" in feature_folder:
        label_index = -2  #because last column of auDeep feature is the label
    else:
        label_index = -1
    if train_file.endswith('.arff'):
        with open(train_file) as f:
            arff_data = arff.load(f)
            train_X = np.array(arff_data['data'])[:, 1:label_index].astype(np.float32)
            train_y = np.array(arff_data['data'])[:, label_index].astype(str)
            feature_names = [attribute[0] for attribute in arff_data['attributes'][1:label_index]]

        with open(devel_file) as f:
            arff_data = arff.load(f)
            devel_names = np.array(arff_data['data'])[:, 0]
            devel_X = np.array(arff_data['data'])[:, 1:label_index].astype(np.float32)
            devel_y = np.array(arff_data['data'])[:, label_index].astype(str)

        with open(test_file) as f:
            arff_data = arff.load(f)
            test_names = np.array(arff_data['data'])[:, 0]
            test_X = np.array(arff_data['data'])[:, 1:label_index].astype(np.float32)
            test_y = np.array(arff_data['data'])[:, label_index].astype(str)

    else:
        train_df = pd.read_csv(train_file)
        devel_df = pd.read_csv(devel_file)
        test_df = pd.read_csv(test_file)
        feature_names = list(train_df.columns)[1:label_index]  #label_index of AuDeep is -2, since last column is the label
        
        train_X = train_df.values[:, 1:label_index].astype(np.float32)
        train_y = train_df.values[:, label_index].astype(str)

        devel_names = devel_df.values[:, 0]
        devel_X = devel_df.values[:, 1:label_index].astype(np.float32)
        devel_y = devel_df.values[:, label_index].astype(str)

        test_names = test_df.values[:, 0]
        test_X = test_df.values[:, 1:label_index].astype(np.float32)
        test_y = test_df.values[:, label_index].astype(str)
        
    num_train = train_X.shape[0]
    num_devel = devel_X.shape[0]
    split_indices = np.repeat([-1, 0], [num_train, num_devel])
    split = PredefinedSplit(split_indices)
    
    X = np.append(train_X, devel_X, axis=0) #number of training data
    y = np.append(train_y, devel_y, axis=0) #number of val data
    
    grid_search = GridSearchCV(estimator=PIPELINE, param_grid=GRID, 
                                scoring=make_scorer(recall_score, average='macro'), 
                                n_jobs=-1, cv=split, refit=True, verbose=1, 
                                return_train_score=False)
    
    # fit on data. train -> devel first, then train+devel implicit
    grid_search.fit(X, y)
    best_estimator = grid_search.best_estimator_
    
    # fit clone of best estimator on train again for devel predictions
    estimator = clone(best_estimator, safe=False)
    estimator.fit(train_X, train_y)
    preds = estimator.predict(devel_X)

    metrics = {'dev': {}, 'test': {}}

    # devel metrics
    run_predictions_devel(estimator, feature_folder, results_folder)
    # uar = recall_score(devel_y, preds, average='macro')
    # cm = confusion_matrix(devel_y, preds)
    # print(f'UAR: {uar}\n{classification_report(devel_y, preds)}\n\nConfusion Matrix:\n\n{cm}') 
    # metrics['dev']['uar'] = uar
    # metrics['dev']['cm'] = cm.tolist()
    # metrics['params'] = make_dict_json_serializable(grid_search.best_params_)

    # df_predictions = pd.DataFrame({'filename': devel_names.tolist(), 'prediction': preds.tolist()})
    # df_predictions.to_csv(os.path.join(results_folder, 'devel.predictions.csv'), index=False)

    pd.DataFrame(grid_search.cv_results_).to_csv(
        os.path.join(results_folder, 'grid_search.csv'), index=False)

    # test metrics
    # print(f'Generating test predictions for optimised parameters {metrics["params"]}')
    preds = best_estimator.predict(test_X)
    if len(set(test_y)) > 1: # test labels exist
        uar = recall_score(test_y, preds, average='macro')
        cm = confusion_matrix(test_y, preds)
        metrics['test']['uar'] = uar
        metrics['test']['cm'] = cm.tolist()
        print(f'UAR: {uar}\n{classification_report(test_y, preds)}\n\nConfusion Matrix:\n\n{cm}') 

        print('Computing CI...')
        uars = list(Parallel(n_jobs=-1, verbose=10)(delayed(bootstrap)(best_estimator, X, y, test_X, test_y, i) for i in range(params['bootstrap_iterations'])))
        ci_low, ci_high = scipy.stats.t.interval(params['ci_interval'], len(uars)-1, loc=np.mean(uars), scale=scipy.stats.sem(uars))
        metrics['test']['ci_low'] = ci_low
        metrics['test']['ci_high'] = ci_high
        metrics['test']['uar_mean'] = np.mean(uars)
        

    df_predictions = pd.DataFrame({'filename': test_names.tolist(), 'prediction': preds.tolist()})
    df_predictions.to_csv(os.path.join(results_folder, 'test.predictions.csv'), index=False)

    with open(os.path.join(results_folder, 'metrics.json'), 'w') as f:
        json.dump(metrics, f)
    
    return estimator
    # feature ranking
    # labels = sorted(set(devel_y))
    # if len(set(devel_y)) < 3:
    #     df = pd.DataFrame(data={'feature': feature_names, f'importance_for_{labels[1]}': best_estimator.named_steps.estimator.coef_[0, :]})
    # else:
    #     df = pd.DataFrame(data={**{'feature': feature_names}, **{f'importance_for_{labels[i]}': best_estimator.named_steps.estimator.coef_[i, :] for i in range(len(labels))}})
    # df.to_csv(os.path.join(results_folder, 'ranking.csv'), index=False)


In [ ]:
params = {}
with open('params.yaml') as f:
    params = yaml.load(f, Loader=yaml.FullLoader)
    params = params['svm']

feature_type = 'deepspectrum'
feature_base = f'{FEATURE_BASE}/{feature_type}'
result_base = f'./results/{feature_type}'
for dirpath, dirnames, filenames in os.walk(feature_base):
    if not dirnames:
        file_extension = os.path.splitext(filenames[0])[1]
        result_dir = os.path.join(result_base, os.path.relpath(dirpath, start=feature_base))
        os.makedirs(result_dir, exist_ok=True)
        model = run_svm(dirpath, result_dir, params)

Fitting 1 folds for each of 40 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  9.2min finished
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max

0.5022199453551912
[[119  64]
 [ 31  17]]
